# Reactive covering

## Introduction

<p>Este documento .ipynb contiene separados los códigos de las diferentes stages del proyecto.<br> 
1) Importación de las librerías y definición de la función 'showSolution', empleada en los diferentes stages.<br>
2) Stage 1: Familiarización con el trabajo, las variables de estudio y primera aproximación al calculo del coste y calidad de los resultados.<br>
3) Stage 2: Método de optimización MILP para optimizar el coste.<br>
4) Stage 3: Método de optimización MILP para optimizar el coste.<br></p>


In [1]:
from matplotlib import pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cf
import math
from docplex.mp.model import Model

import random
import json

def showSolution(data,solution):
    
    # print the booking strategy
    print("Bookings:")
    for booking in solution['Bookings']:
        print(booking)

    # show the passes
    minVisiDuration = data['minVisiDuration']
    AOIs = data['AOIs']
    nAOIs = len(AOIs)
    accesses = data['Accesses']
    satPasses = data['Passes']
    nPasses = len(satPasses)
    
    ncols = 3
    nrows = math.ceil(nPasses / ncols)
    proj = ccrs.PlateCarree()
    fig, axs = plt.subplots(nrows=nrows,ncols=ncols, subplot_kw={'projection': proj},figsize=(30,80))
    axs = axs.flatten()

    for i in range(nPasses):
        satPass = satPasses[i]
        ax = axs[i]
        ax.set_extent([-15, 25, 35, 60])
        ax.stock_img()
        ax.add_feature(cf.COASTLINE, lw=1)
        ax.add_feature(cf.BORDERS)
        #plt.gcf().set_size_inches(15, 15)    
        for aoi in  AOIs:
            ax.plot([aoi['lon']], [aoi['lat']], 'kx')
        ax.set_title("Pass #{} [{},{}]".format(i,satPass['startDate'],satPass['endDate']))
    
        booking = None
        for bk in solution['Bookings']:
            if bk['passId'] == i:
                booking = bk            
                break
        
        for accessId in satPass['accessIds']:
            access = accesses[accessId]
            aoi = AOIs[access['aoiId']]            
            ax.plot([aoi['lon']], [aoi['lat']], 'rx')
            # show the reservations
            if (booking != None) and (booking['bookingStart'] <= access['end'] - minVisiDuration) and (access['start'] + minVisiDuration <= booking['bookingEnd']):
                ax.plot([aoi['lon']], [aoi['lat']], 'bo')

    plt.show()

## Stage 1

Explicar como hemos calculado el coste y como hemos definido la quality y meter un poco de paja rollo intro
Coste: Numero de obs por el coste fijo + coste de observacion por coste variable [coste/s]
Obtencion de la calidad es el numero de observaciones satifechas sobre numero de obs requeridas. Cada obs satisfecha suma 1 y se divide por el total de observaciones. Accesos del satelite en observacion request y que satelte se conecte durante la duracion de la observacion request.

In [15]:
# Import the data
fileName = "data2"
dataFile = open('data/{0}.json'.format(fileName))  
data = json.load(dataFile)

# Load data
obsRequests = data['ObservationRequests']
fixedPassCost = data['fixedPassCost']
minVisiDuration = data['minVisiDuration']
aois = data['AOIs']
accesses = data['Accesses']
referenceDate = data['referenceDate']
passCostPerTimeUnit = data['passCostPerTimeUnit']
goals = data['Goals']
satPasses = data['Passes']

# Lengths of lists
nAois = len(aois)
nObsRequests = len(obsRequests)
nObsRequestsToCover = nObsRequests
nPasses = len(satPasses)
nGoals = len(goals)

# Very basic solver
# We start considering all the AOIs uncovered
covered = [False for i in range(nObsRequests)]
remainingSatPassesIds = list(range(nPasses))
random.shuffle(remainingSatPassesIds)
selectedPassIds = []
for i in remainingSatPassesIds:
    # Try to select the ith path
    satPass = satPasses[i]
    keepPass = False
    for accessId in satPass['accessIds']:
        access = accesses[accessId]
        for j in range(nObsRequests):
            obsRequest = obsRequests[j]
            if not covered[j] and accessId in obsRequest['accessIds']:
                covered[j] = True
                keepPass = True
                nObsRequestsToCover -= 1
                
    if keepPass:
        selectedPassIds.append(i)
            
    if nObsRequestsToCover == 0:
        break

# Computation of the total cost
timeUses = [satPasses[id]['end'] - satPasses[id]['start'] for id in selectedPassIds]
totalCost = len(selectedPassIds)*data['fixedPassCost'] + sum(timeUses)*data['passCostPerTimeUnit']

# Computation of the quality of the covering: It is defined as the relation of the 
# requests covered over the total amount of requests
quality = 0
for aoiId in range(nAois):
    aux = 0
    for goalId in range(nGoals):
        for step in range(goals[goalId]['nSteps']):
            obsReqId = aoiId + nAois*aux + step + (goals[goalId]['nSteps'] - 1)*aoiId
            requestStart = step*goals[goalId]['duStep']
            requestEnd = step*goals[goalId]['duStep'] + goals[goalId]['rctHorizon']
            for passId in selectedPassIds:
                if any(accessIdObsReq in satPasses[passId]['accessIds'] for accessIdObsReq in obsRequests[obsReqId]['accessIds']):
                    if (requestStart<=satPasses[passId]['end']-30) and (requestEnd>=satPasses[passId]['start']+30):
                        quality +=1
                        break
        aux += goals[goalId]['nSteps']
        
quality /= nObsRequests

# Export the solution to json
bookings = []
for i in selectedPassIds:
    satPass = satPasses[i]
    bookings.append({"passId": i, "passStart": satPass['start'], "passEnd": satPass['end'], "bookingStart": satPass['start'], "bookingEnd": satPass['end']})
jsonString = json.dumps({"Bookings": bookings})
jsonFile = open("mySolutionStage1{0}.json".format(fileName), "w")
jsonFile.write(jsonString)
jsonFile.close()

# Presentation of the results
print("------------------------------RESULTS------------------------------")
print("Total cost of the covering: {0}".format(totalCost))
print("Quality of the covering: {0}".format(quality))
print("Bookings:")
for booking in bookings:
    print(booking)

------------------------------RESULTS------------------------------
Total cost of the covering: 19002
Quality of the covering: 0.984516129032258
Bookings:
{'passId': 12, 'passStart': 8318, 'passEnd': 8612, 'bookingStart': 8318, 'bookingEnd': 8612}
{'passId': 47, 'passStart': 30900, 'passEnd': 30965, 'bookingStart': 30900, 'bookingEnd': 30965}
{'passId': 8, 'passStart': 5443, 'passEnd': 5599, 'bookingStart': 5443, 'bookingEnd': 5599}
{'passId': 51, 'passStart': 33359, 'passEnd': 33629, 'bookingStart': 33359, 'bookingEnd': 33629}
{'passId': 30, 'passStart': 20266, 'passEnd': 20500, 'bookingStart': 20266, 'bookingEnd': 20500}
{'passId': 55, 'passStart': 35635, 'passEnd': 35701, 'bookingStart': 35635, 'bookingEnd': 35701}
{'passId': 26, 'passStart': 17649, 'passEnd': 17994, 'bookingStart': 17649, 'bookingEnd': 17994}
{'passId': 3, 'passStart': 1624, 'passEnd': 1740, 'bookingStart': 1624, 'bookingEnd': 1740}
{'passId': 36, 'passStart': 24624, 'passEnd': 24666, 'bookingStart': 24624, 'bookin

## Stage 2

Optimizar coste sin tiempo variable

In [18]:
# Import the data
fileName = "data4"
dataFile = open('data/{0}.json'.format(fileName))  
data = json.load(dataFile)

# Load data
obsRequests = data['ObservationRequests']
fixedPassCost = data['fixedPassCost']
minVisiDuration = data['minVisiDuration']
aois = data['AOIs']
accesses = data['Accesses']
referenceDate = data['referenceDate']
passCostPerTimeUnit = data['passCostPerTimeUnit']
goals = data['Goals']
satPasses = data['Passes']

# Lengths of lists
nAois = len(aois)
nObsRequests = len(obsRequests)
nPasses = len(satPasses)
nGoals = len(goals)

# OPTIMISATION MODEL
# Model creation
model = Model("reactiveCovering")

# Variables
selectedPass = model.binary_var_list(range(nPasses), name='selectedPass')

# Objectif:
# 1) Minimisation of the cost
model.minimize(model.sum(selectedPass)*data['fixedPassCost']
    + model.sum((satPasses[id]['end'] - satPasses[id]['start'])*selectedPass[id] 
    for id in range(nPasses))*data['passCostPerTimeUnit'])

# Contraints:
# 1) Ensuring that all the goals are covered bu at least one access of a pass
def covering(requestStart, requestEnd, passId, obsReqId):
    if any(accessIdObsReq in satPasses[passId]['accessIds'] for accessIdObsReq in obsRequests[obsReqId]['accessIds']):
        if requestStart<=(satPasses[passId]['end'] - 30) and requestEnd>=(satPasses[passId]['start'] + 30):
            return selectedPass[passId]
    return 0

# Checking for all the observation requests
for aoiId in range(nAois):
    aux = 0
    for goalId in range(nGoals):
        for step in range(goals[goalId]['nSteps']):
            obsReqId = aoiId + nAois*aux + step + (goals[goalId]['nSteps'] - 1)*aoiId
            requestStart = step*goals[goalId]['duStep']
            requestEnd = step*goals[goalId]['duStep'] + goals[goalId]['rctHorizon']
            coverings = [covering(requestStart, requestEnd, passId, obsReqId) for passId in range(nPasses)]
            sum_covering = model.sum(coverings)
            if not str(sum_covering) == "0":
                model.add_constraint(sum_covering >= 1)
        aux += goals[goalId]['nSteps']

# Solve:
# Computation time limit
model.set_time_limit(300)

model.print_information()

# Call of the solver
solution = model.solve(log_output=True)

# Getting the solution
selectedPassIds = []
if solution is None:
    raise Exception("No solution")
else:
    for i in range(nPasses):
        if selectedPass[i].solution_value == 1:
            selectedPassIds.append(i)

# Computation of the total cost
timeUses = [satPasses[id]['end'] - satPasses[id]['start'] for id in selectedPassIds]
totalCost = len(selectedPassIds)*data['fixedPassCost'] + sum(timeUses)*data['passCostPerTimeUnit']

# Computation of the quality of the covering: It is defined as the relation of the 
# requests covered over the total amount of requests
quality = 0
for aoiId in range(nAois):
    aux = 0
    for goalId in range(nGoals):
        for step in range(goals[goalId]['nSteps']):
            obsReqId = aoiId + nAois*aux + step + (goals[goalId]['nSteps'] - 1)*aoiId
            requestStart = step*goals[goalId]['duStep']
            requestEnd = step*goals[goalId]['duStep'] + goals[goalId]['rctHorizon']
            for passId in selectedPassIds:
                if any(accessIdObsReq in satPasses[passId]['accessIds'] for accessIdObsReq in obsRequests[obsReqId]['accessIds']):
                    if (requestStart<=satPasses[passId]['end']-30) and (requestEnd>=satPasses[passId]['start']+30):
                        quality +=1
                        break
        aux += goals[goalId]['nSteps']
        
quality /= nObsRequests

# Export the solution to json
bookings = []
for i in selectedPassIds:
    satPass = satPasses[i]
    bookings.append({"passId": i, "passStart": satPass['start'], "passEnd": satPass['end'], "bookingStart": satPass['start'], "bookingEnd": satPass['end']})
jsonString = json.dumps({"Bookings": bookings})
jsonFile = open("mySolutionStage2{0}.json".format(fileName), "w")
jsonFile.write(jsonString)
jsonFile.close()

# Presentation of the results
print("------------------------------RESULTS------------------------------")
print("Total cost of the covering: {0}".format(totalCost))
print("Quality of the covering: {0}".format(quality))
print("Bookings:")
for booking in bookings:
    print(booking)


Model: reactiveCovering
 - number of variables: 69
   - binary=69, integer=0, continuous=0
 - number of constraints: 2234
   - linear=2234
 - parameters:
     parameters.timelimit = 300.00000000000000
 - objective: minimize
 - problem type is: MILP
Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
CPXPARAM_RandomSeed                              202001241
CPXPARAM_TimeLimit                               300
Found incumbent of value 22621.000000 after 0.00 sec. (0.07 ticks)
Tried aggregator 3 times.
MIP Presolve eliminated 2214 rows and 48 columns.
MIP Presolve modified 44 coefficients.
Aggregator did 2 substitutions.
Reduced MIP has 18 rows, 19 columns, and 81 nonzeros.
Reduced MIP has 19 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (3.12 ticks)
Probing time = 0.00 sec. (0.01 ticks)
Tried aggregator 1 time.
Reduced MIP has 18 rows, 19 columns, and 81 nonzeros.
Reduced MIP has 19 binaries, 0 generals, 0 

## Stage 3

Tiempo variable.

In [21]:
# Import the data
fileName = "data4"
dataFile = open('data/{0}.json'.format(fileName)) 
data = json.load(dataFile)

# Load data
obsRequests = data['ObservationRequests']
fixedPassCost = data['fixedPassCost']
minVisiDuration = data['minVisiDuration']
aois = data['AOIs']
accesses = data['Accesses']
referenceDate = data['referenceDate']
passCostPerTimeUnit = data['passCostPerTimeUnit']
goals = data['Goals']
satPasses = data['Passes']

# Lengths of lists
nAois = len(aois)
nObsRequests = len(obsRequests)
nPasses = len(satPasses)
nGoals = len(goals)

# OPTIMISATION MODEL
# Model creation
model = Model("reactiveCovering")

# Variables

# Definition of the boundary lists
lbs = []
ube = []
for i in range(nPasses):
    lbs.append(satPasses[i]['start'])
    ube.append(satPasses[i]['end'])
    
selectedPass = model.binary_var_list(range(nPasses), name='selectedPass')
tStart = model.continuous_var_list(range(nPasses), lb=lbs, name='tStart')
tEnd = model.continuous_var_list(range(nPasses), ub=ube, name='tEnd')


# Objectif:
# 1) Minimisation of the cost
model.minimize(model.sum(selectedPass)*data['fixedPassCost'] 
               + (model.sum((tEnd[id] - tStart[id])for id in range(nPasses)) 
                  - 30*(nPasses - model.sum(selectedPass)))*data['passCostPerTimeUnit'])

# TEST
#model.minimize(model.sum(selectedPass)*data['fixedPassCost']
 #   + model.sum(duration[id]*selectedPass[id] 
  #  for id in range(nPasses))*data['passCostPerTimeUnit'])
#for passId in range(nPasses):
 #   model.add_constraint(duration[passId] == (tEnd[passId] - tStart[passId]))    
  

# Contraints:
for passId in range(nPasses):
    model.add_constraint((tEnd[passId] - tStart[passId]) >= 30)
    
# 1) Ensuring that all the goals are covered bu at least one access of a pass
def accessing(requestStart, requestEnd, passId, obsReqId):
    if any(accessIdObsReq in satPasses[passId]['accessIds'] for accessIdObsReq in obsRequests[obsReqId]['accessIds']):
        #print('Access of the observation request in the pass {0}'.format(passId))
        if requestStart<=(satPasses[passId]['end'] - 30) and requestEnd>=(satPasses[passId]['start'] + 30):
            return selectedPass[passId]
    return 0

# Checking for all the observation requests
for aoiId in range(nAois):
    aux = 0
    for goalId in range(nGoals):
        for step in range(goals[goalId]['nSteps']):
            obsReqId = aoiId + nAois*aux + step + (goals[goalId]['nSteps'] - 1)*aoiId
            requestStart = step*goals[goalId]['duStep']
            requestEnd = step*goals[goalId]['duStep'] + goals[goalId]['rctHorizon']
            accessings = [accessing(requestStart, requestEnd, passId, obsReqId) for passId in range(nPasses)]
            sum_accessing = model.sum(accessings)
            if not str(sum_accessing) == "0":
                # 1) Ensuring that all the goals are accessed
                model.add_constraint(sum_accessing >= 1)
                for passId in range(nPasses):
                    if not str(accessings[passId]) == "0":
                        # 2) tStart covering constraint
                        #model.add_constraint(requestStart <= tStart[passId])
                        model.add_constraint(requestStart <= tEnd[passId]-30)
                        # 3) tEnd covering constraint
                        #model.add_constraint(requestEnd >= tEnd[passId])
                        model.add_constraint(requestEnd >= tStart[passId]+30)
        aux += goals[goalId]['nSteps']


# Solve
# Time limitation
model.set_time_limit(60)

model.print_information()

# Call of the solver
solution = model.solve(log_output=True)

# solution.display()

# Getting the solution
selectedPassIds = []
if solution is None:
    raise Exception("No solution")
else:
    for i in range(nPasses):
        if selectedPass[i].solution_value == 1:
            selectedPassIds.append(i)
            


# Computation of the total cost
timeUses = [tEnd[id].solution_value - tStart[id].solution_value for id in selectedPassIds]
totalCost = len(selectedPassIds)*data['fixedPassCost'] + model.sum(timeUses)*data['passCostPerTimeUnit']

# Computation of the quality of the covering: It is defined as the relation of the 
# requests covered over the total amount of requests
quality = 0
for aoiId in range(nAois):
    aux = 0
    for goalId in range(nGoals):
        for step in range(goals[goalId]['nSteps']):
            obsReqId = aoiId + nAois*aux + step + (goals[goalId]['nSteps'] - 1)*aoiId
            requestStart = step*goals[goalId]['duStep']
            requestEnd = step*goals[goalId]['duStep'] + goals[goalId]['rctHorizon']
            for passId in selectedPassIds:
                if any(accessIdObsReq in satPasses[passId]['accessIds'] for accessIdObsReq in obsRequests[obsReqId]['accessIds']):
                    if (requestStart<=tEnd[passId].solution_value-30) and (requestEnd>=tStart[passId].solution_value+30):
                        quality +=1
                        break
        aux += goals[goalId]['nSteps']
        
quality /= nObsRequests

# export the solution to json
bookings = []
for i in selectedPassIds:
    satPass = satPasses[i]
    bookings.append({"passId": i, "passStart": satPass['start'], "passEnd": satPass['end'], "bookingStart": tStart[i].solution_value, "bookingEnd": tEnd[i].solution_value})
jsonString = json.dumps({"Bookings": bookings})
jsonFile = open("mySolutionStage3{0}.json".format(fileName), "w")
jsonFile.write(jsonString)
jsonFile.close()

# Presentation of the results
print("------------------------------RESULTS------------------------------")
print("Total cost of the covering: {0}".format(totalCost))
print("Quality of the covering: {0}".format(quality))
print("Bookings:")
for booking in bookings:
    print(booking)


Model: reactiveCovering
 - number of variables: 207
   - binary=69, integer=0, continuous=138
 - number of constraints: 30527
   - linear=30527
 - parameters:
     parameters.timelimit = 60.00000000000000
 - objective: minimize
 - problem type is: MILP
Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
CPXPARAM_RandomSeed                              202001241
CPXPARAM_TimeLimit                               60
Tried aggregator 2 times.
MIP Presolve eliminated 30523 rows and 203 columns.
Aggregator did 4 substitutions.
All rows and columns eliminated.
Presolve time = 0.01 sec. (14.60 ticks)

Root node processing (before b&c):
  Real time             =    0.01 sec. (15.91 ticks)
Parallel b&c, 8 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.01 sec. (15.91 ticks)
------------

## Stage 4: Multi-criteria optimisation

EXPLICAR

In [88]:
# Import the data
fileName = "data2"
dataFile = open('data/{0}.json'.format(fileName))  
data = json.load(dataFile)

# Load data
obsRequests = data['ObservationRequests']
fixedPassCost = data['fixedPassCost']
minVisiDuration = data['minVisiDuration']
aois = data['AOIs']
accesses = data['Accesses']
referenceDate = data['referenceDate']
passCostPerTimeUnit = data['passCostPerTimeUnit']
goals = data['Goals']
satPasses = data['Passes']

# Lengths of lists
nAois = len(aois)
nObsRequests = len(obsRequests)
nPasses = len(satPasses)
nGoals = len(goals)

alpha = 1.0

# OPTIMISATION MODEL
# Model creation
model = Model("reactiveCovering")

# Variables
# # Definition of the boundary lists
lbs = []
ube = []
for i in range(nPasses):
    lbs.append(satPasses[i]['start'])
    ube.append(satPasses[i]['end'])
    
selectedPass = model.binary_var_list(range(nPasses), name='selectedPass')
#tStart = model.continuous_var_list(range(nPasses), lb=lbs, name='tStart')
#tEnd = model.continuous_var_list(range(nPasses), ub=ube, name='tEnd')

# Objectif:
# 1) Minimisation of the cost and maximisation of the quality
# 1.1) Cost
#cost = (model.sum(selectedPass)*data['fixedPassCost'] 
 #              + (model.sum((tEnd[id] - tStart[id])for id in range(nPasses)) 
  #                - 30*(nPasses - model.sum(selectedPass)))*data['passCostPerTimeUnit'])

cost = (model.sum(selectedPass)*data['fixedPassCost']
    + model.sum((satPasses[id]['end'] - satPasses[id]['start'])*selectedPass[id] 
    for id in range(nPasses))*data['passCostPerTimeUnit'])

maxcost = 0
for id in range(nPasses):
    maxcost += fixedPassCost + (satPasses[id]['end'] - satPasses[id]['start'])*data['passCostPerTimeUnit'] 

# 1.2°
quality = 0
for aoiId in range(nAois):
    aux = 0
    for goalId in range(nGoals):
        for step in range(goals[goalId]['nSteps']):
            obsReqId = aoiId + nAois*aux + step + (goals[goalId]['nSteps'] - 1)*aoiId
            requestStart = step*goals[goalId]['duStep']
            requestEnd = step*goals[goalId]['duStep'] + goals[goalId]['rctHorizon']
            for i in range(len(selectedPass)):
                if any(accessIdObsReq in satPasses[i]['accessIds'] for accessIdObsReq in obsRequests[obsReqId]['accessIds']):
                    quality += selectedPass[i]
                    break
        aux += goals[goalId]['nSteps']
        
quality /= nObsRequests

model.minimize(alpha*cost/maxcost+(1-alpha)*(1-quality))

# Contraints:
#for passId in range(nPasses):
 #   model.add_constraint((tEnd[passId] - tStart[passId]) >= 30)

# Solve
# Time limitation
model.set_time_limit(60)

model.print_information()

# Call of the solver
solution = model.solve(log_output=True)

# solution.display()

# Getting the solution
selectedPassIds = []
if solution is None:
    raise Exception("No solution")
else:
    for i in range(nPasses):
        if selectedPass[i].solution_value == 1:
            selectedPassIds.append(i)

# Computation of the total cost
# timeUses = [tEnd[id].solution_value - tStart[id].solution_value for id in selectedPassIds]
timeUses = [satPasses[id]['end'] - satPasses[id]['start'] for id in selectedPassIds]
totalCost = len(selectedPassIds)*data['fixedPassCost'] + model.sum(timeUses)*data['passCostPerTimeUnit']

# Computation of the quality of the covering: It is defined as the relation of the 
# requests covered over the total amount of requests
quality = 0
for aoiId in range(nAois):
    aux = 0
    for goalId in range(nGoals):
        for step in range(goals[goalId]['nSteps']):
            obsReqId = aoiId + nAois*aux + step + (goals[goalId]['nSteps'] - 1)*aoiId
            requestStart = step*goals[goalId]['duStep']
            requestEnd = step*goals[goalId]['duStep'] + goals[goalId]['rctHorizon']
            for passId in selectedPassIds:
                if any(accessIdObsReq in satPasses[passId]['accessIds'] for accessIdObsReq in obsRequests[obsReqId]['accessIds']):
                     if (requestStart<=satPasses[passId]['end']-30) and (requestEnd>=satPasses[passId]['start']+30):
                        #if (requestStart<=tEnd[passId].solution_value-30) and (requestEnd>=tStart[passId].solution_value+30):
                        quality +=1
                        break
        aux += goals[goalId]['nSteps']
        
quality /= nObsRequests

# export the solution to json
bookings = []
for i in selectedPassIds:
    satPass = satPasses[i]
    # bookings.append({"passId": i, "passStart": satPass['start'], "passEnd": satPass['end'], "bookingStart": tStart[i].solution_value, "bookingEnd": tEnd[i].solution_value})
    bookings.append({"passId": i, "passStart": satPass['start'], "passEnd": satPass['end'], "bookingStart": satPass['start'], "bookingEnd": satPass['end']})
jsonString = json.dumps({"Bookings": bookings})
jsonFile = open("mySolutionStage4{0}Alpha{1}.json".format(fileName, int(alpha*100)), "w")
jsonFile.write(jsonString)
jsonFile.close()

# Presentation of the results
print("------------------------------RESULTS------------------------------")
print("Total cost of the covering: {0}".format(totalCost))
print("Quality of the covering: {0}".format(quality))
print("Bookings:")
for booking in bookings:
    print(booking)

Model: reactiveCovering
 - number of variables: 69
   - binary=69, integer=0, continuous=0
 - number of constraints: 0
   - linear=0
 - parameters:
     parameters.timelimit = 60.00000000000000
 - objective: minimize
 - problem type is: MILP
Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
CPXPARAM_RandomSeed                              202001241
CPXPARAM_TimeLimit                               60
Found incumbent of value 0.000000 after 0.00 sec. (0.00 ticks)

Root node processing (before b&c):
  Real time             =    0.00 sec. (0.00 ticks)
Parallel b&c, 8 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.00 sec. (0.00 ticks)
------------------------------RESULTS------------------------------
Total cost of the covering: 0
Quality of the covering: 0.0
Bookings:


## Stage 4: Glutton algorithm

EXPLICAR

## Solution presentation

EXPLICAR

In [25]:
stage = 4
fileName = "data12"
alpha = 0.5

# Import the data
dataFile = open('data/{0}.json'.format(fileName))  
data = json.load(dataFile)

# Load data
obsRequests = data['ObservationRequests']
fixedPassCost = data['fixedPassCost']
minVisiDuration = data['minVisiDuration']
aois = data['AOIs']
accesses = data['Accesses']
referenceDate = data['referenceDate']
passCostPerTimeUnit = data['passCostPerTimeUnit']
goals = data['Goals']
satPasses = data['Passes']

# Lengths of lists
nAois = len(aois)
nObsRequests = len(obsRequests)
nPasses = len(satPasses)
nGoals = len(goals)

# Import the solution
if stage == 4:
    solutionFile = open("mySolutionStage4{0}Alpha{1}.json".format(fileName, int(alpha*100)))  
else:
    solutionFile = open("mySolutionStage{0}{1}.json".format(stage,fileName))  
solution = json.load(solutionFile)

# Reading of the solution
bookings = solution["Bookings"]
selectedPassIds = []
tStart = [0] * nPasses
tEnd = [0] * nPasses
for i in range(len(bookings)):
    selectedPassIds.append(bookings[i]['passId'])
    tStart[bookings[i]['passId']] = bookings[i]['bookingStart']
    tEnd[bookings[i]['passId']] = bookings[i]['bookingEnd']

# Computation of the total cost
timeUses = [tEnd[id] - tStart[id] for id in selectedPassIds]
totalCost = len(selectedPassIds)*data['fixedPassCost'] + sum(timeUses)*data['passCostPerTimeUnit']

# Computation of the quality of the covering: It is defined as the relation of the 
# requests covered over the total amount of requests
quality = 0
for aoiId in range(nAois):
    aux = 0
    for goalId in range(nGoals):
        for step in range(goals[goalId]['nSteps']):
            obsReqId = aoiId + nAois*aux + step + (goals[goalId]['nSteps'] - 1)*aoiId
            requestStart = step*goals[goalId]['duStep']
            requestEnd = step*goals[goalId]['duStep'] + goals[goalId]['rctHorizon']
            for passId in selectedPassIds:
                if any(accessIdObsReq in satPasses[passId]['accessIds'] for accessIdObsReq in obsRequests[obsReqId]['accessIds']):
                    #if (requestStart<=satPasses[passId]['end']-30) and (requestEnd>=satPasses[passId]['start']+30):
                    if (requestStart<=tEnd[passId]-30) and (requestEnd>=tStart[passId]+30):
                        quality +=1
                        break
        aux += goals[goalId]['nSteps']
        
quality /= nObsRequests

# Presentation of the results
print("IDs of the {0} selected passes: {1}".format(len(selectedPassIds), selectedPassIds))
print("Total cost of the covering: {0}".format(totalCost))
print("Quality of the covering: {0}".format(quality))

showSolution(data,solution)

FileNotFoundError: [Errno 2] No such file or directory: 'mySolutionStage4data12Alpha0.json'